In [ ]:
]activate ../..

### Includes, Drawing Utils:

In [ ]:
using Gen
using GenWorldModels

In [ ]:
include("../main.jl")
AI = AudioInference

In [ ]:
function img_with_tones(img, tones, val=100)
  img = copy(img)
  for tone in tones
    add_tone_to_img!(img, tone; val=val)
  end
  return img
end

function add_tone_to_img!(img, tone; val=100)
  (region, erb) = tone
  yval = Int(floor(AudioInference.pos_for_erb_val(erb)))
  for i=region[1]:region[2]
    img[yval, i] = val
  end
end

In [ ]:
function add_vert_bars(img, xvals; val=100)
  img = copy(img)
  for xval in xvals
    add_vert_bar!(img, xval, val=val)
  end
  img
end
function add_vert_bar!(img, xval; val=100)
    img[:, xval] .= val
end

In [ ]:
function draw_rect!(img, ((x1, y1), (x2, y2)); color=100)
  for x=x1:min(x2, size(img)[1])
    img[x, min(y1, size(img)[2])] = color
    img[x, min(y2, size(img)[2])] = color
  end
  for y=y1:min(y2, size(img)[2])
    img[min(x1, size(img)[1]), y] = color
    img[min(x2, size(img)[1]), y] = color
  end
end
function img_with_rects(img, rects; color=100)
  img = copy(img)
  for rect in rects
    draw_rect!(img, rect; color=color)
  end
  return img
end

In [ ]:
function add_vert_segs(img, segs; val=100)
  img = copy(img)
  for seg in segs
    add_vert_seg!(img, seg, val=val)
  end
  img
end
function add_vert_seg!(img, seg; val=100)
    for y=seg.ymin:min(seg.ymax, size(img)[1])
      img[y, seg.x] = val
  end
end

### Testing Detector

In [ ]:
include("../detector.jl")

Util to visualize source(s):

In [ ]:
function img_with_sources(img, sources, scenelength; val=100)
  img = copy(img)
  for s in sources
    add_source_to_img!(img, s, scenelength)
  end
  img
end
function add_source_to_img!(img, source, scenelength; val=100)
  if source.is_noise
    add_noise_to_img!(img, source, scenelength)
  else
    add_tone_to_img!(img, source, scenelength)
  end
end
function add_tone_to_img!(img, source, scenelength; val=100)
  y = Int(floor(Detector.pos_for_erb_val(source.amp_or_erb)))
  xwidth = size(img)[2]
  start = Int(floor(source.onset/scenelength * xwidth))
  dur = Int(floor(source.duration/scenelength * xwidth))
  for x=start:min(start + dur, xwidth)
    img[y, x] = val
  end
end
function add_noise_to_img!(img, source, scenelength; val=100)
  xwidth = size(img)[2]
  start = Int(floor(source.onset/scenelength * xwidth))
  dur = Int(floor(source.duration/scenelength * xwidth))
  draw_rect!(img, ((1, start), (size(img)[1], min(start + dur, xwidth))); color=val)
end

In [ ]:
function rect_for_source(img, source, scenelength)
  TONESIZE = 10
  xwidth = size(img)[2]
  start = Int(floor(source.onset/scenelength * xwidth))
  dur = Int(floor(source.duration/scenelength * xwidth))
  xmin, xmax = start, min(start + dur, xwidth)
  if !source.is_noise
    y = Int(floor(Detector.pos_for_erb_val(source.amp_or_erb)))
    ymin, ymax = max(1, y-Int(TONESIZE/2)), min(size(img)[1], y+Int(TONESIZE/2))
  else
    ymin, ymax = 1, size(img)[1]
  end
  return ((ymin, xmin), (ymax, xmax))
end

In [ ]:
function img_with_source_rects(img, sources, scenelength; val=100)
  img = copy(img)
  for s in sources
    draw_rect!(img, rect_for_source(img, s, scenelength); color=val)
  end
  img
end

Test rectangle detecting:

In [ ]:
ground_truth, _ = generate(AI.generate_scene, AI.args, choicemap((:kernel => :n_tones, 4)))
initial_tr, weight = AI.generate_initial_tr(ground_truth, num_sources=0)
gram = AI.error_gram(initial_tr)
AI.plot_gtg(img_with_rects(gram, Iterators.flatten(Detector.get_noise_tone_rects(gram; threshold=0.3))))

In [ ]:
sources = Detector.get_detected_sources(gram; threshold=0.3, scenelength=2.)
AI.plot_gtg(img_with_source_rects(gram, sources, 2))

In [ ]:
s = Detector.get_noise_tone_rects(gram; threshold=0.4)[2][2]
AI.plot_gtg(img_with_rects(gram, (s,)))

In [ ]:
s

In [ ]:
AI.plot_gtg(img_with_sources(gram, (Detector.tonesource(s, gram, 2.),), 2))

In [ ]:
((miny, minx), (maxy, maxx)) = s
println("mean y: ", (miny + maxy) / 2)
erb = Detector.tone_erb_for_val((miny + maxy)/2)
println("erb: ", erb)
println("back to yval: ", Detector.pos_for_erb_val(erb))

In [ ]:
λ = 2

In [ ]:
"""
    score(gram, s::Source; threshold=0.5)

The score of a birth action gives a heuristic estimate of how "good" it is to add this object.
This score is calculated by inspecting the rough rectangle in which the tone/noise will be placed,
and counting how many pixels in the errorgram are "on", minus how many are off.  We normalize this by dividing
by the rectangle area, to get a measure of "fraction_improvement"
(what fraction of the area is an improvement, minus the fraction that makes things worse).
The score is e^(λ * improvement).
"""
function score(gram, s::Detector.Source; threshold=0.5)
    ((miny, minx), (maxy, maxx)) = rect_for_source(gram, s, 2)
    gram = @view gram[miny:maxy, minx:maxx]
    maxval = maximum(gram)
    greater_than_threshold = gram .> maxval * threshold
    less_than_threshold = gram .< maxval * threshold
    area = (maxx - minx) * (maxy - miny)
    improvement = (sum(greater_than_threshold) - sum(less_than_threshold))/area
    return exp(λ * improvement)
end


In [ ]:
for source in Detector.get_detected_sources(gram; threshold=0.3, scenelength=2)
  println("SCORE: ", score(gram, source; threshold=0.3))
  AI.plot_gtg(img_with_source_rects(gram, (source,), 2))
  AI.PyPlot.figure()
end